<a href="https://colab.research.google.com/github/VThuan5421/DataAnalyst/blob/main/learning/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
path = "https://raw.githubusercontent.com/VThuan5421/DataAnalyst/main/data/intents.json"

In [20]:
import nltk
nltk.download("punkt")
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
import json
import pandas as pd

In [16]:
words = [] # store all words in the patterns in intents file
classes = [] # store type of patterns
documents = [] # store all words (patterns) and it's classes
ignore_words = ["?", "!"]
# read file
intents = pd.read_json(path)

In [12]:
intents['intents']

0     {'tag': 'greeting', 'patterns': ['Hi there', '...
1     {'tag': 'goodbye', 'patterns': ['Bye', 'See yo...
2     {'tag': 'thanks', 'patterns': ['Thanks', 'Than...
3     {'tag': 'noanswer', 'patterns': [], 'responses...
4     {'tag': 'options', 'patterns': ['How you could...
5     {'tag': 'adverse_drug', 'patterns': ['How to c...
6     {'tag': 'blood_pressure', 'patterns': ['Open b...
7     {'tag': 'blood_pressure_search', 'patterns': [...
8     {'tag': 'search_blood_pressure_by_patient_id',...
9     {'tag': 'pharmacy_search', 'patterns': ['Find ...
10    {'tag': 'search_pharmacy_by_name', 'patterns':...
11    {'tag': 'hospital_search', 'patterns': ['Looku...
12    {'tag': 'search_hospital_by_params', 'patterns...
13    {'tag': 'search_hospital_by_type', 'patterns':...
Name: intents, dtype: object

In [17]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    # tokenize pattern to words
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [21]:
# stem words (create base word). Remove duplicate
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [23]:
print("Length (words): ", len(words), " ", words[:5])
print("Length (classes): ", len(classes), " ", classes)

Length (words):  88   ["'s", ',', 'a', 'adverse', 'all']
Length (classes):  9   ['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']


In [31]:
#normalize text to binary data
training = []
output_empty = [0] * len(classes) # expected output is a one of classes
for doc in documents:
  pattern = doc[0]
  # initialize bag of words for training
  bag = []
  for w in words:
    bag.append(1) if w in pattern else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])

np.random.shuffle(training)


In [32]:
training = np.array(training)
print(training[:1, 0])
print(training[:1, 1])

[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])]
[list([0, 0, 0, 1, 0, 0, 0, 0, 0])]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [33]:
# Create x_train, y_train
x_train = list(training[:, 0])
y_train = list(training[:, 1])
# define the model
model = Sequential()
model.add(Dense(128, activation = "relu", input_shape = (len(x_train[0]), )))
model.add(Dropout(0.5))

model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))

model.add(Dense(len(y_train[0]), activation = "softmax"))
# compiling and fitting model
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])
hist = model.fit(np.array(x_train), np.array(y_train), verbose = 1, epochs = 200, batch_size = 5)
#model.save("model.h5", hist)
print("Done")

Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 0.6796 - accuracy: 0.1489
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 0.6341 - accuracy: 0.1277
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 0.6091 - accuracy: 0.1489
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 0.5661 - accuracy: 0.1915
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 0.5279 - accuracy: 0.1489
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 0.4881 - accuracy: 0.1489
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 0.4572 - accuracy: 0.1064
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 0.4152 - accuracy: 0.2553
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 0.3913 - accuracy: 0.2340
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 0.3831 - accuracy: 0.1915
Epoch 11/

In [37]:
score = model.evaluate(x_train, y_train)
print("Loss: ", score[0])
print("Accuracy: ", score[1])

2/2 [==============================] - 0s 5ms/step - loss: 0.0275 - accuracy: 0.9362
Loss:  0.02750740945339203
Accuracy:  0.936170220375061


In [38]:
# Model prediction
# some text for prediction
text = ["hello", "what can you do", "thanks", "drug", "bye", "do you love me"]

In [43]:
def clean_up_sentence(sentence):
  word = nltk.word_tokenize(sentence)
  # lemmatize each word, create base word
  pattern_word = [lemmatizer.lemmatize(w.lower()) for w in word]
  return pattern_word

# normailize text to binary data
def text_to_binary(sentence, words):
  pattern_word = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for s in pattern_word:
    for i, w in enumerate(words):
      if s == w:
        bag[i] = 1
  return bag

# Predict class
def predict_class(sentence, words, classes):
  bag = text_to_binary(sentence, words)
  res = model.predict(np.array([bag]))[0]
  idx_argmax = np.argmax(res)
  print("Result: ", idx_argmax, res[idx_argmax])
  return classes[idx_argmax], res[idx_argmax] # class and probability

# get response
def get_response(sentence, words, classes):
  tag, prob = predict_class(sentence, words, classes)
  intents_list = intents['intents']
  result = ""
  for i in intents_list:
    if i['tag'] == tag:
      result = np.random.choice(i["responses"])
      break
  return result

In [44]:
for i in text:
  print("Human: ", i)
  print("Bot: ", get_response(i, words, classes))

Human:  hello
Result:  0 0.6534842
Bot:  Hello, thanks for asking
Human:  what can you do
Result:  3 0.9999995
Bot:  Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies
Human:  thanks
Result:  2 0.9949025
Bot:  Any time!
Human:  drug
Result:  4 0.94296265
Bot:  Navigating to Adverse drug reaction module
Human:  bye
Result:  1 0.94459295
Bot:  See you!
Human:  do you love me
Result:  3 0.9774943
Bot:  I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies
